In [313]:
import numpy as np
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier,export_graphviz
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
import os
import subprocess
from sklearn import tree
from sklearn  import linear_model
from io import StringIO
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import  cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import average_precision_score
from operator import itemgetter , attrgetter, methodcaller
from sklearn.svm import LinearSVC
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

In [314]:
d = pd.read_csv("C:/Users/Dell/Downloads/CrimePredictionData1/CrimePredictionData/communities-crime-clean.csv")
d['highcrime'] = d['ViolentCrimesPerPop'].map(lambda x: True if x > 0.1 else False)

# 1. Decision Trees

a.)	Create a new field “highCrime” which is true if the crime rate per capita (ViolentCrimesPerPop) is greater than 0.1, and false otherwise. What are the percentage of positive and negative instances in the dataset?

In [315]:
mylist=d['highcrime'].value_counts()
true_percent = (mylist[1]/(d['highcrime'].shape))*100
false_percent =(mylist[0]/(d['highcrime'].shape))*100
print("true percetage is %2f " %true_percent)
print("false percentage is %2f "%false_percent)

true percetage is 62.719518 
false percentage is 37.280482 


b.	 Use DecisionTreeClassifier to learn a decision tree to predict highCrime on the entire dataset (remember to exclude the crime rate feature from the input feature set so you are not cheating). 

In [316]:
d1= d.copy()
del d1['state']
del d1['communityname']
del d1['fold']
del d1['highcrime']
del d1['ViolentCrimesPerPop']


 i)What are the training accuracy, precision, and recall for this tree? 
 
--->The training accuracy is 1,precision is 1,and recall is 1 for this tree.

In [317]:
clf = tree.DecisionTreeClassifier()

without_split_clf = clf.fit(d1,d['highcrime'])
prediction = without_split_clf.predict(fil1)
print("Accuracy: %2f" %clf.score(fil1,d['highcrime']))
print(precision_recall_fscore_support(d['highcrime'],prediction))

Accuracy: 1.000000
(array([ 1.,  1.]), array([ 1.,  1.]), array([ 1.,  1.]), array([ 743, 1250], dtype=int64))


ii)What are the main features used for classification? Can you explain why they make sense (or not)?

-->The main features used for classification is  PctKids2Par,racePctWhite,racePctHisp,PctLess9thGrade.........The classification makes sense because the more important the feature is the higher it is,the importance of the feature is computed as the reduction of the criterion brought by that feature this is called as the Gini importance.The important features are population of kids with two parents.This feature has heighest weight according to decision tree and this implies that kids with 2 parents are often vulnerable to and lack good parenting leading which implies the reason they are involved in crime to fulfill their need.



In [318]:
my_features = d1.columns
#print(my_features)
imp_feature = clf.feature_importances_
indices = np.argsort(imp_feature)[::-1]
print ('Feature Ranking') 
for i in range(10):
    print ("{} feature {} ({})".format(i+1,my_features[indices[i]],imp_feature[indices[i]]))
    
#index = np.argsort(imp_feature)[::-1]
#print ('Ranking of features as:') 
#for i in range(10):
    #print(my_features(index))
    #print ("{} Feature no.{} ({})".format(i+1,imp_feature[i],imp_feature[index[i]] ))
 

Feature Ranking
1 feature PctKids2Par (0.36008283807486524)
2 feature racePctWhite (0.08753352660889117)
3 feature racePctHisp (0.04512009456234423)
4 feature PctLess9thGrade (0.022254287471314773)
5 feature PctEmplManu (0.019610344222201464)
6 feature PctImmigRec10 (0.015830355918708735)
7 feature PctHousOwnOcc (0.015116201414033855)
8 feature HispPerCap (0.013342308151642607)
9 feature MedRent (0.013290038119924881)
10 feature PctEmploy (0.012654683296951364)


c.	Now apply cross-validation (cross_val_score) to do 10-fold cross-validation to estimate the out-of-training accuracy of decision tree learning for this task.
        
i.	What are the 10-fold cross-validation accuracy, precision, and recall?
        
--->The 10-fold cross-validation accuracy is 0.725497,precision is 0.786514,recall is 0.770400.



In [319]:
print("Precision is  %f" % cross_val_score(clf , d1, d['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall is  %f" %cross_val_score(clf , d1, d['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy is %f" %cross_val_score(clf , d1, d['highcrime'], cv=10, scoring='accuracy' ).mean())

Precision is  0.793351
Recall is  0.770400
Accuracy is 0.725492


ii.	Why are they different from the results in the previous test?

--->The results are different from the previous test because in the cross-validation we are having different test and training dataset,while in the first case we are testing the same data we are training it which leads to overfitting.The precision,accuracy and recall is 1,whereas for this precision,recall and accuracy is 0.78,0.77,0.72 which is a little lesser.


# 2. Linear Classification



 a.	Use GaussianNB to learn a Naive Bayes classifier to predict highCrime.

 i.	What is the 10-fold cross-validation accuracy, precision, and recall for this method?
 
 --->The 10-fold cross-validation accuracy is 0.761608,precision is 0.911800,recall is 0.692000.


In [320]:
clf2= GaussianNB()
print("Precision is  %f" % cross_val_score(clf2 , d1, d['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall is  %f" %cross_val_score(clf2 , d1, d['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy is %f" %cross_val_score(clf2 , d1, d['highcrime'], cv=10, scoring='accuracy' ).mean())

Precision is  0.911800
Recall is  0.692000
Accuracy is 0.761608


ii)What are the 10 most predictive features? This can be measured by the normalized absolute difference of means for the feature between the two classes:
|μ_T-μ_F |/(σ_T+σ_F )

-->The most predictive features are listed below.The mean for the feature between the two classes depicts that the more the mean the better it has classified the two classes.


In [321]:
d3 =  d1.copy()
clf2.fit(d1,d['highcrime'])
d3['predict'] = clf2.predict(d1)
d3

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,predict
0,0.01,0.61,0.21,0.83,0.02,0.01,0.41,0.49,0.26,0.21,...,0.03,0.70,0.40,0.34,0.57,0.05,0.06,0.01,0.00,False
1,0.01,0.41,0.55,0.57,0.01,0.00,0.47,0.45,0.31,0.57,...,0.00,0.93,0.66,0.82,0.84,0.11,0.03,0.01,0.00,True
2,0.03,0.34,0.86,0.30,0.04,0.01,0.41,0.42,0.27,0.59,...,0.04,0.77,0.59,0.70,0.64,0.06,0.11,0.04,0.00,True
3,0.01,0.38,0.35,0.71,0.04,0.01,0.39,0.46,0.31,0.49,...,0.03,0.78,0.56,0.67,0.71,0.09,0.05,0.00,0.00,False
4,0.04,0.37,0.32,0.70,0.21,0.02,1.00,1.00,1.00,0.14,...,0.12,0.49,0.12,0.00,0.15,0.09,0.09,0.01,0.00,True
5,0.04,0.44,1.00,0.10,0.00,0.00,0.43,0.45,0.30,0.58,...,0.00,0.96,0.74,0.95,0.89,0.11,0.07,0.13,0.00,True
6,0.41,0.37,1.00,0.02,0.03,0.01,0.41,0.50,0.33,0.48,...,0.03,0.90,0.64,0.86,0.82,0.43,0.15,0.20,0.38,True
7,0.01,0.30,0.00,0.99,0.02,0.01,0.38,0.37,0.27,0.74,...,0.00,0.82,0.54,0.72,0.76,0.04,0.07,0.01,0.00,False
8,0.00,0.39,0.31,0.75,0.02,0.02,0.35,0.43,0.22,0.33,...,0.03,0.60,0.46,0.40,0.54,0.03,0.08,0.01,0.00,False
9,0.06,0.39,0.32,0.73,0.04,0.01,0.39,0.46,0.27,0.39,...,0.03,0.71,0.47,0.61,0.60,0.14,0.09,0.01,0.00,False


   Printing top 10 features using NBgaussian

In [322]:
#NaiveBaisGaussian

In [323]:
predict_true_d = d3[d3['predict'] == True]
predict_false_d = d3[fil3['predict'] == False]


In [324]:
Nb = []
for i in range(len(d1.columns)):
    Nb.append((abs(predict_true_d[[i]].mean() - predict_false_d[[i]].mean()) / predict_true_d[[i]].std() + predict_false_d[[i]].std() ))
sorted(Nb, key = itemgetter(0), reverse= True)

[PctKids2Par    1.881312
 dtype: float64, pctWInvInc    1.841672
 dtype: float64, PctPersOwnOccup    1.769371
 dtype: float64, medFamInc    1.763879
 dtype: float64, PctFam2Par    1.757952
 dtype: float64, medIncome    1.733898
 dtype: float64, FemalePctDiv    1.679538
 dtype: float64, PctYoungKids2Par    1.59169
 dtype: float64, PctHousLess3BR    1.579829
 dtype: float64, TotalPctDiv    1.575829
 dtype: float64, PctHousOwnOcc    1.529413
 dtype: float64, PctTeen2Par    1.5067
 dtype: float64, PctPopUnderPov    1.477284
 dtype: float64, perCapInc    1.463695
 dtype: float64, MalePctDivorce    1.371709
 dtype: float64, pctWPubAsst    1.362389
 dtype: float64, racePctWhite    1.348088
 dtype: float64, PctUnemployed    1.29353
 dtype: float64, PctIlleg    1.292631
 dtype: float64, PctNotHSGrad    1.278326
 dtype: float64, PctOccupMgmtProf    1.270628
 dtype: float64, PctHousNoPhone    1.266779
 dtype: float64, blackPerCap    1.235194
 dtype: float64, HispPerCap    1.226781
 dtype: float64

The larger this different, the more predictive the feature. Why do these make sense (or not)?

The answer  to this question is that we can increase the difference of mean or we can decrease the sum of standard deviations. 

iii.	How do these results compare with your results from decision trees, above?
the answer to this question is:

--->When compared with the results from decision trees it can be implied that NaiveBais Classifier is better classifier when compared to decision tree because the accuracy and recall is better for NBGaussian classifier when compared to decision tree. 
    
    



    
    

# Use LinearSVC 

use linear SVC to learn a linear Support Vector Machine model to predict highCrime.

Linear Support vector classification(SVC) is similar to SVC when the kernel parameter ='linear'  

In [325]:

clf3 = LinearSVC()

i.	 What is the 10-fold cross-validation accuracy, precision, and recall for this method?

--->The 10-fold cross-validation accuracy is 0.796234,precision is 0.845405 and recall is 0.834400.

In [326]:
print("Precision is %f" % cross_val_score(clf3 , d1, d['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall is  %f" %cross_val_score(clf3 , d1, d['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy is %f" %cross_val_score(clf3 , d1, d['highcrime'], cv=10, scoring='accuracy' ).mean())

Precision is 0.845405
Recall is  0.834400
Accuracy is 0.796234


ii.	What are the 10 most predictive features? 

In [327]:
clf3.fit(d1,d['highcrime'])
total_feature = d1.columns
important_feature = clf3.coef_[0]
#important_feature
indices = np.argsort(important_feature)[::-1]
print ('Feature Ranking') 
for i in range(10):
    print ("{} feature no.{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))


Feature Ranking
1 feature no.PersPerOccupHous (1.7551275649516747)
2 feature no.RentHighQ (1.0668849551067916)
3 feature no.MalePctDivorce (1.0656965630774555)
4 feature no.NumUnderPov (1.0515481865360268)
5 feature no.NumStreet (1.019155327194038)
6 feature no.PctOccupMgmtProf (1.0146715964715038)
7 feature no.population (1.0023017243838481)
8 feature no.agePct12t21 (0.9882724517210235)
9 feature no.HousVacant (0.9365131453676593)
10 feature no.agePct65up (0.890572635141866)


This can be measured by the absolute feature weights in the model. Why do these make sense (or not)?
this makes sense because:

iii.How do these results compare with your results from decision trees, above?
--->In Linear SVC, it has more flexible in the selection of loss functions and penalties,henceforth SVC would evaluate better with large number of samples.When looked at the accuracy,precision and recall we can imply that linear SVC is better over decision trees.

# 3. Regression:

a.	Use LinearRegression to learn a linear model directly predicting the crime rate per capita (ViolentCrimesPerPop).

In [328]:
regr = linear_model.LinearRegression()
regr.fit(d1,d['ViolentCrimesPerPop'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

i.	Using 10-fold cross-validation, what is the estimated mean-squared-error (MSE) of the model?
The  estimated mean-squared-error (MSE) of the model is -0.020093969304445319.MSE is the mean value of the squared deviations of the predictions from the true values when compared to an out-of-sample test space genearted by a model estimated over a particular sample space.

In [329]:
prediction = regr.predict(d1)
mse_scores = cross_val_score(regr, d1 ,d['ViolentCrimesPerPop'], scoring = 'neg_mean_squared_error' , cv=10)
mse_scores.mean()

-0.020093969304445319

ii.	What is the MSE on the training set (train on all the data then test on it all)?

--->The MSE on the training set is 0.016516774880307179.

In [330]:
mean_squared_error(d['ViolentCrimesPerPop'], prediction)


0.016516774880307179

iii.	What features are most predictive of a high crime rate? A low crime rate?

---> The most predictive of a high crime rate and low crime rate are illustarted below:
This is based on that all the features are on the same scale ,the most important features should have the highest coefficient in the model,while featured unrelated with the output variables should have coefficient values close to zero.



In [331]:
total_feature = d1.columns
important_feature = regr.coef_
#important_feature
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature no.{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))
print(" ")
for i in range(10):
    print ("{} Top 10 low feature no.{} ({})".format(i+1,total_feature[index[i]],important_feature[index[i]]))


Feature Ranking
1 Top 10 high feature no.PersPerOccupHous (0.6350881164986149)
2 Top 10 high feature no.PctHousOwnOcc (0.5681332098865255)
3 Top 10 high feature no.MalePctDivorce (0.45851704864166676)
4 Top 10 high feature no.PctRecImmig8 (0.43251055765291013)
5 Top 10 high feature no.MedRent (0.3727277977113885)
6 Top 10 high feature no.medFamInc (0.2879788749460697)
7 Top 10 high feature no.PctEmploy (0.248474316084703)
8 Top 10 high feature no.MalePctNevMarr (0.22672791284180477)
9 Top 10 high feature no.PctPersDenseHous (0.2143525641419764)
10 Top 10 high feature no.OwnOccMedVal (0.21287586365359717)
 
1 Top 10 low feature no.PctPersOwnOccup (-0.6756944788028594)
2 Top 10 low feature no.TotalPctDiv (-0.561924314414664)
3 Top 10 low feature no.whitePerCap (-0.3510157744407789)
4 Top 10 low feature no.PctKids2Par (-0.32265127649627473)
5 Top 10 low feature no.OwnOccLowQuart (-0.3081702191931843)
6 Top 10 low feature no.numbUrban (-0.29644254317756596)
7 Top 10 low feature no.PersPerR

# Ridge Regression


b.	Now use Ridge regression to reduce the amount of overfitting, using RidgeCV to pick the best alpha from among (10, 1, 0.1, 0.01, and 0.001).
    
    i.	What is the estimated MSE of the model under 10-fold CV?
    
    --->The estiamted MSE of the model under 10-fold CV is -0.019795021348168747.


In [332]:
ridge = linear_model.RidgeCV(alphas = (10, 1, 0.1, 0.01, 0.001))
ridge.fit(d1, d['ViolentCrimesPerPop'])
mse = cross_val_score(ridge, d1, d['ViolentCrimesPerPop'], scoring = 'neg_mean_squared_error' ,cv=10)
mse.mean()

-0.019795021348168747

ii.	What is the MSE on the training set (train on all the data then test on it all)?

--->The MSE on the training set (train on all the data then test on it all) is 0.016763529155169436

In [333]:
ridge_predict = ridge.predict(d1)
mean_squared_error(fil['ViolentCrimesPerPop'], ridge_predict)

0.016763529155169436

iii.	What is the best alpha?

--->The best Alpha is 1.0 because Alpha is also called as regularization,which is used to set certain penality so that the noise over the data is reduced.If the alpha value is very large then the regularization effect dominates the squared loss function and the coefficients tends to be zero.If the alpha value tends to be zero and the solution implies towards the ordinary least squares then coefficients tend to be big oscillation.Henceforth,it is preferable that the alpha is selected such that it maintains the balance between both. 

In [334]:
ridge.alpha_

1.0

iv.	What does this say about the amount of overfitting in linear regression for this problem?

  ---> In linear regression the results will be based on some estimates it is modeled from the training data coefficients of every variable used which will lead to overfitting.Not each declared coefficient estimates will tell the true patterns and relationship in dataset.In rideg regression,penality is added and would provide us prior knowledge about how the regression should or should not play out which is determined by alpha value.We can observe that the MSE is almost similar for both which implies that there is no overfitting i.e linear Regression -0.020093969304445319 ,ridge Regression  -0.019795021348168747.MSE can be considered to determine whether the model is over or under-fitted.
    	
    

# c.polynomial features

c.	Now use polynomial features to do quadratic (second-order) polynomial regression.

i.	What is the estimated MSE of the model under 10-fold CV?

--->The estimated MSE of the model under 10-fold CV is -0.131116.


In [335]:
features = d[list(d)].values
labels = d1.values
features_x = np.array(features)
labels_y = np.array(labels)

featuresReg = d1[list(d1)].values
labelsReg = d['ViolentCrimesPerPop'].values
featuresReg_x = np.array(featuresReg)
labelsReg_y = np.array(labelsReg)
labelsReg_y

clf4 = PolynomialFeatures(degree=2)
X_ = clf4.fit_transform(featuresReg_x)


clf_poly = linear_model.LinearRegression()
clf_poly.fit(X_, labelsReg_y)
predict_ = clf4.fit_transform(featuresReg_x)
y_pred = clf_poly.predict(predict_)


In [336]:
#mse = cross_val_score(ridge, fil1, fil['ViolentCrimesPerPop'], scoring = 'neg_mean_squared_error' ,cv=10)
#mse.mean()
msr_reg = cross_val_score(clf_poly, featuresReg_x,  labelsReg_y, cv=10, scoring='neg_mean_squared_error')
print(" mean %f" % cross_val_score(clf_poly , d1, d['highcrime'] , cv=10, scoring='neg_mean_squared_error' ).mean())

 mean -0.131116


ii.	What is the MSE on the training set (train on all the data then test on it all)?

--->The MSE on the training set is 1.69580219538e-28.

In [337]:
mse1=mean_squared_error(d['ViolentCrimesPerPop'],y_pred)
print("Mean squared error for the training set is: ")
print(mse1)


Mean squared error for the training set is: 
1.69580219538e-28


iii.	Does this mean the quadratic model is better than the linear model for this problem?

--->From the data we can imply that the MSE values for both quadratic and linear model is similar with a little difference,henceforth we can depict that they both are giving similar results.When we draw the curve it can be observed that both quadratic and linear model are linear in nature.If the values varied for the polynomial model it would be because of the data point are more covered for polynomial curve.


# 4.	Dirty Data

Repeat the decision tree learning question for the full (non-clean) data set and present the results.

In [338]:
dirty =  pd.read_csv("C:/Users/Dell/Downloads/CrimePredictionData1/CrimePredictionData/communities-crime-full.csv")
dirty['highcrime'] = dirty['ViolentCrimesPerPop'].map(lambda x: True if x > 0.1 else False)

In [339]:
dirty_1 = dirty.drop(['state','communityname','fold','ViolentCrimesPerPop','county','community','highcrime'], axis=1) 

In [340]:
dirty

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop,highcrime
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20,True
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.12,0.45,?,?,?,?,0.00,?,0.67,True
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.21,0.02,?,?,?,?,0.00,?,0.43,True
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.39,0.28,?,?,?,?,0.00,?,0.12,True
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.09,0.02,?,?,?,?,0.00,?,0.03,False
5,6,?,?,SouthPasadenacity,1,0.02,0.28,0.06,0.54,1.00,...,0.58,0.10,?,?,?,?,0.00,?,0.14,True
6,44,7,41500,Lincolntown,1,0.01,0.39,0.00,0.98,0.06,...,0.08,0.06,?,?,?,?,0.00,?,0.03,False
7,6,?,?,Selmacity,1,0.01,0.74,0.03,0.46,0.20,...,0.33,0.00,?,?,?,?,0.00,?,0.55,True
8,21,?,?,Hendersoncity,1,0.03,0.34,0.20,0.84,0.02,...,0.17,0.04,?,?,?,?,0.00,?,0.53,True
9,29,?,?,Claytoncity,1,0.01,0.40,0.06,0.87,0.30,...,0.47,0.11,?,?,?,?,0.00,?,0.15,True


In [341]:
#evrything replaced my mean
for i in dirty_1.columns:
    dirty_1[[i]] = dirty_1[[i]].convert_objects(convert_numeric=True)
    dirty_1[[i]] = dirty_1[[i]].fillna(dirty_1[[i]].mean())

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [342]:
dirty_1

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.290000,0.12,0.26,0.20,0.060000,0.040000,0.900000,0.500000,0.32,0.140000
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.305987,0.02,0.12,0.45,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.305987,0.01,0.21,0.02,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.305987,0.02,0.39,0.28,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.305987,0.04,0.09,0.02,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
5,0.02,0.28,0.06,0.54,1.00,0.25,0.31,0.48,0.27,0.37,...,0.305987,0.01,0.58,0.10,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
6,0.01,0.39,0.00,0.98,0.06,0.02,0.30,0.37,0.23,0.60,...,0.305987,0.05,0.08,0.06,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
7,0.01,0.74,0.03,0.46,0.20,1.00,0.52,0.55,0.36,0.35,...,0.305987,0.01,0.33,0.00,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
8,0.03,0.34,0.20,0.84,0.02,0.00,0.38,0.45,0.28,0.48,...,0.305987,0.04,0.17,0.04,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
9,0.01,0.40,0.06,0.87,0.30,0.03,0.90,0.82,0.80,0.39,...,0.305987,0.00,0.47,0.11,0.163103,0.076708,0.698589,0.440439,0.00,0.195078


In [343]:
c1= tree.DecisionTreeClassifier()

print("Precision is  %f" % cross_val_score(cl , dirty_1, dirty['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall is  %f" %cross_val_score(cl , dirty_1, dirty['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy is  %f" %cross_val_score(cl , dirty_1, dirty['highcrime'], cv=10, scoring='accuracy' ).mean())
c1.fit(dirty_1, dirty['highcrime'])
without_split_clf = clf.fit(d1,d['highcrime'])
prediction = without_split_clf.predict(fil1)
print("Accuracy: %2f" %clf.score(fil1,d['highcrime']))
print(precision_recall_fscore_support(d['highcrime'],prediction))
important_feature = c1.feature_importances_
#important_feature
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

Precision is  0.870938
Recall is  0.867308
Accuracy is  0.835997
Accuracy: 1.000000
(array([ 1.,  1.]), array([ 1.,  1.]), array([ 1.,  1.]), array([ 743, 1250], dtype=int64))
Feature Ranking
1 Top 10 high feature arePctKids2Par (0.35879596764301586)
2 Top 10 high feature areracePctWhite (0.08791053427053004)
3 Top 10 high feature areracePctHisp (0.04857087695880409)
4 Top 10 high feature arePctEmplManu (0.023213080619819686)
5 Top 10 high feature arePctLess9thGrade (0.018582907773771345)
6 Top 10 high feature areMedRent (0.014853225070985195)
7 Top 10 high feature arePctImmigRecent (0.014769926832739767)
8 Top 10 high feature arePctImmigRec8 (0.014056975339810356)
9 Top 10 high feature arePctWOFullPlumb (0.01300474528677165)
10 Top 10 high feature areHispPerCap (0.012856798093584471)


a.	Are the CV results better or worse? What does this say about the effect of missing values?

--->The CV results are better because the precision,recall and accuracy is more for dirty data than clean data.
We have considered taking mean because when we plot the gaussian distribution we get normal distribution.
 
    

 #5.	Team(Extra credit)

In [344]:
i.Experiment with two learning methods other than those described above (one can be a non-linear kernel for SVM) for the classification problem, explaining clearly what you did. Show CV results for both the clean and full datasets.

SyntaxError: invalid syntax (<ipython-input-344-794b42413ae0>, line 1)

In [ ]:
#we will be using first as logistic Regression:
#for clean_data


cl = linear_model.LogisticRegression()

cl.fit(d1, d['highcrime'])

rs1 = cross_val_score(cl, d1, d['highcrime'], cv=10)
print(rs1)
print ("Accuracy %f" %cross_val_score(cl , d1, d['highcrime'], cv=10, scoring='accuracy' ).mean())

important_feature = cl.coef_[0]
#important_feature
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

In [ ]:
#for dirty data
cl_dirty = linear_model.LogisticRegression()
cl_dirty.fit(d1, d['highcrime'])
rs1_dirty = cross_val_score(cl_dirty, dirty_1, dirty['highcrime'], cv=10)
print(rs1_dirty)
print ("Accuracy %f" %cross_val_score(cl_dirty , dirty_1, dirty['highcrime'], cv=10, scoring='accuracy' ).mean())

important_feature = cl_dirty.coef_[0]
#important_feature
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

Logistic regression is more robust 

In [ ]:
# using random Forest for clean data



In [ ]:
clf_Random = RandomForestClassifier(n_jobs=2)
clf_Random.fit(d1, d['highcrime'])

rs1 = cross_val_score(cl, d1, d['highcrime'], cv=10)
print(rs1)
print ("Accuracy %f" %cross_val_score(clf_Random , d1, d['highcrime'], cv=10, scoring='accuracy' ).mean())


# Build a classification task using 3 informative features
important_feature = clf_Random.feature_importances_
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))


In [ ]:
#using Random Frest for dirty data
clRandom_dirty = RandomForestClassifier(n_jobs=2)
clRandom_dirty.fit(dirty_1, dirty['highcrime'])
rsrandom_dirty = cross_val_score(clRandom_dirty, dirty_1, dirty['highcrime'], cv=10)
print(rsrandom_dirty)
print ("Accuracy %f" %cross_val_score(clRandom_dirty , dirty_1, dirty['highcrime'], cv=10, scoring='accuracy' ).mean())

important_feature = clRandom_dirty.feature_importances_
#important_feature
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

Random forest have been observed to overfit for few data.For category type of data random forest is not preferable.

 While we consider the clean data and dirty data for  logistic regtression is better for dirty data .  Logistic regression is better when compared to random forest as it gives easier probability scores and gives efficient implementation.In case of random forest the model is generally observed to overfit for some datasets with noisy classification which is not in case of logistic regression thus providing better.Random forest tends to be biased for the categorical form of datsset with different levels which is the case here.
